In [8]:
# Load CSV into SQLite
import pandas as pd
import sqlite3
from pathlib import Path

raw_csv = Path("../Data/raw/ER Wait Time Dataset.csv")
db_file = Path("../Data/processed/er_visits.db")

# load
df = pd.read_csv(raw_csv)

# sanitize column names
def clean_col(c):
    return (c.strip()
             .lower()
             .replace(' ', '_')
             .replace('(', '')
             .replace(')', '')
             .replace('-', '_')
             .replace('__','_'))

df.columns = [clean_col(c) for c in df.columns]

# quick inspect
print(df.columns.tolist())
print(df.shape)
print(df.head(2))

# write sqlite (replace if exists)
conn = sqlite3.connect(db_file)
df.to_sql("er_visits", conn, if_exists="replace", index=False)
conn.close()
print("wrote", db_file)

['visit_id', 'patient_id', 'hospital_id', 'hospital_name', 'region', 'visit_date', 'day_of_week', 'season', 'time_of_day', 'urgency_level', 'nurse_to_patient_ratio', 'specialist_availability', 'facility_size_beds', 'time_to_registration_min', 'time_to_triage_min', 'time_to_medical_professional_min', 'total_wait_time_min', 'patient_outcome', 'patient_satisfaction']
(5000, 19)
               visit_id patient_id hospital_id                 hospital_name  \
0  HOSP-1-20240210-0001  PAT-00001      HOSP-1  Springfield General Hospital   
1  HOSP-3-20241128-0001  PAT-00002      HOSP-3  Northside Community Hospital   

  region           visit_date day_of_week  season   time_of_day urgency_level  \
0  Urban  2024-02-10 20:20:56    Saturday  Winter  Late Morning        Medium   
1  Rural  2024-11-28 02:07:47    Thursday    Fall       Evening        Medium   

   nurse_to_patient_ratio  specialist_availability  facility_size_beds  \
0                       4                        3             

In [17]:
import sqlite3, pandas as pd
conn = sqlite3.connect("../Data/processed/er_visits.db")

# 1) confirm row count
pd.read_sql_query("SELECT COUNT(*) AS cnt FROM er_visits", conn)

# 2) check schema (columns + types)
pd.read_sql_query("PRAGMA table_info('er_visits')", conn)

# 3) sample rows
pd.read_sql_query("SELECT * FROM er_visits LIMIT 5", conn)

conn.close()

In [11]:
import sqlite3
import pandas as pd

# path to SQLite DB
db_file = "../Data/processed/er_visits.db"

# create a connection object
conn = sqlite3.connect(db_file)

# SQL query to count missing values
missing_query = """
SELECT
    SUM(CASE WHEN visit_id IS NULL OR trim(visit_id) = '' THEN 1 ELSE 0 END) AS visit_id_missing,
    SUM(CASE WHEN patient_id IS NULL OR trim(patient_id) = '' THEN 1 ELSE 0 END) AS patient_id_missing,
    SUM(CASE WHEN hospital_id IS NULL THEN 1 ELSE 0 END) AS hospital_id_missing,
    SUM(CASE WHEN total_wait_time_min IS NULL THEN 1 ELSE 0 END) AS total_wait_missing,
    SUM(CASE WHEN nurse_to_patient_ratio IS NULL THEN 1 ELSE 0 END) AS nurse_ratio_missing,
    SUM(CASE WHEN patient_satisfaction IS NULL THEN 1 ELSE 0 END) AS patient_satisfaction_missing
FROM er_visits;
"""

# run query and load into pandas
missing_df = pd.read_sql_query(missing_query, conn)
print(missing_df)

# close connection
conn.close()



   visit_id_missing  patient_id_missing  hospital_id_missing  \
0                 0                   0                    0   

   total_wait_missing  nurse_ratio_missing  patient_satisfaction_missing  
0                   0                    0                             0  


In [ ]:
# Average Wait Time by Urgency Level

import sqlite3
import pandas as pd

# path to SQLite DB
db_file = "../Data/processed/er_visits.db"

# create a connection object
conn = sqlite3.connect(db_file)

# SQL query to calculate average wait time by urgency level
avg_wait_query = """
SELECT urgency_level,
       ROUND(AVG(total_wait_time_min),2) AS avg_wait_time,
       COUNT(*) AS visit_count
FROM er_visits
GROUP BY urgency_level
order BY avg_wait_time DESC;
"""
# run query and load into pandas
avg_wait_df = pd.read_sql_query(avg_wait_query, conn)
print(avg_wait_df)



  urgency_level  avg_wait_time  visit_count
0           Low         173.54         1222
1        Medium          93.70         1291
2          High          43.19         1245
3      Critical          18.35         1242


In [17]:
# Peak Times (Average Wait by Hour of Day)

# SQL query to calculate average wait time by hour of day
peak_time_query = """
SELECT STRFTIME('%H', visit_date) AS hour_of_day,
       ROUND(AVG(total_wait_time_min),2) AS avg_wait_time,
       COUNT(*) AS visit_count
FROM er_visits
GROUP BY hour_of_day
ORDER BY hour_of_day;
"""
# run query and load into pandas
peak_time_df = pd.read_sql_query(peak_time_query, conn)
print(peak_time_df)



   hour_of_day  avg_wait_time  visit_count
0           00          82.46          228
1           01          72.75          218
2           02          84.73          194
3           03          84.65          171
4           04          82.62          182
5           05          82.40          219
6           06          80.86          213
7           07          77.07          206
8           08          89.24          221
9           09          90.04          215
10          10          75.85          231
11          11          81.01          228
12          12          84.28          197
13          13          83.08          212
14          14          80.49          213
15          15          75.77          199
16          16          78.20          219
17          17          83.57          181
18          18          79.78          205
19          19          82.51          217
20          20          86.24          205
21          21          89.05          227
22         

In [19]:
# Day of Week Patterns
# SQL query to calculate average wait time by day of week
day_of_week_query = """
SELECT day_of_week,
       ROUND(AVG(total_wait_time_min),2) AS avg_wait,
       COUNT(*) AS visit_count
FROM er_visits
GROUP BY day_of_week
ORDER BY avg_wait DESC;
"""
# run query and load into pandas
day_of_week_df = pd.read_sql_query(day_of_week_query, conn)
print(day_of_week_df)


  day_of_week  avg_wait  visit_count
0      Monday    101.58          768
1      Friday     90.39          685
2    Thursday     84.23          706
3     Tuesday     79.08          741
4   Wednesday     78.30          674
5    Saturday     72.14          701
6      Sunday     66.56          725


In [ ]:
# Hospital Ranking by Average Wait Time
# SQL query to rank hospitals by average wait time

ranking_query = """
SELECT hospital_name,
       RouND(AVG(total_wait_time_min),2) AS avg_wait_time,
         COUNT(*) AS visit_count
FROM er_visits
GROUP BY hospital_name
HAVING COUNT(*) >= 100
ORDER BY avg_wait_time DESC
"""
# run query and load into pandas
ranking_df = pd.read_sql_query(ranking_query, conn)
print(ranking_df)


                  hospital_name  avg_wait_time  visit_count
0  Springfield General Hospital          82.70          994
1    St. Mary’s Regional Health          81.89          995
2      Riverside Medical Center          81.81         1023
3  Northside Community Hospital          81.76          999
4          Summit Health Center          81.43          989


In [24]:
# Impact of Nurse-to-Patient Ratio
# SQL query to analyze impact of nurse-to-patient ratio on wait times

nurse_ratio_query = """
SELECT nurse_to_patient_ratio,
       ROUND(AVG(total_wait_time_min),2) AS avg_wait_time,
       COUNT(*) AS visit_count
FROM er_visits
GROUP BY nurse_to_patient_ratio
ORDER BY avg_wait_time ASC;
"""
# run query and load into pandas
nurse_ratio_df = pd.read_sql_query(nurse_ratio_query, conn)
print(nurse_ratio_df)

   nurse_to_patient_ratio  avg_wait_time  visit_count
0                       1          16.08          637
1                       2          20.75          605
2                       3          63.70         1323
3                       4         107.34         1800
4                       5         172.12          635


In [25]:
# Specialist Availability Effect
# SQL query to analyze effect of specialist availability on wait times

specialist_query = """
SELECT specialist_availability,
       ROUND(AVG(total_wait_time_min),2) AS avg_wait_time,
       COUNT(*) AS visit_count
FROM er_visits
GROUP BY specialist_availability;
"""
# run query and load into pandas
specialist_df = pd.read_sql_query(specialist_query, conn)
print(specialist_df)

    specialist_availability  avg_wait_time  visit_count
0                         0          77.16          475
1                         1          79.14          833
2                         2          83.04          798
3                         3          88.85          824
4                         4          80.63          325
5                         5          76.77          289
6                         6          80.35          299
7                         7          86.89          257
8                         8          78.96          285
9                         9          78.74          294
10                       10          84.50          321


In [29]:
# Facility Size vs Wait Time
# SQL query to analyze facility size impact on wait times

facility_size_query = """
SELECT facility_size_beds,
       ROUND(AVG(total_wait_time_min),2) AS avg_wait_time,
       COUNT(*) AS visit_count
FROM er_visits
GROUP BY facility_size_beds
ORDER BY facility_size_beds;
"""
# run query and load into pandas
facility_size_df = pd.read_sql_query(facility_size_query, conn)
print(facility_size_df)


     facility_size_beds  avg_wait_time  visit_count
0                    10          76.89           53
1                    11          51.57           30
2                    12          93.64           56
3                    13          94.38           45
4                    14          83.12           51
..                  ...            ...          ...
186                 196          60.39           23
187                 197          86.50           16
188                 198         125.50           18
189                 199          91.79           19
190                 200          78.57           14

[191 rows x 3 columns]


In [32]:
# Patient Satisfaction vs Wait Times
# SQL query to analyze correlation between patient satisfaction and wait times

satisfaction_query = """
SELECT ROUND(total_wait_time_min/10)*10 AS wait_time_bucket,
         ROUND(AVG(patient_satisfaction),2) AS avg_satisfaction,
         COUNT(*) AS patients
FROM er_visits
GROUP BY wait_time_bucket
ORDER BY wait_time_bucket;
"""
# run query and load into pandas
satisfaction_df = pd.read_sql_query(satisfaction_query, conn)
print(satisfaction_df)


    wait_time_bucket  avg_satisfaction  patients
0                0.0              4.70       138
1               10.0              4.50       688
2               20.0              4.21       550
3               30.0              3.87       443
4               40.0              3.49       356
5               50.0              3.15       304
6               60.0              2.88       243
7               70.0              2.51       237
8               80.0              2.21       207
9               90.0              1.84       216
10             100.0              1.51       166
11             110.0              1.19       162
12             120.0              1.00       167
13             130.0              1.00       145
14             140.0              1.00       147
15             150.0              1.00       118
16             160.0              1.00        97
17             170.0              1.00        95
18             180.0              1.00        90
19             190.0

In [33]:
# Outcome Analysis
# SQL query to analyze outcomes based on wait times

outcome_query = """
Select patient_outcome,
         ROUND(AVG(total_wait_time_min),2) AS avg_wait_time,
         COUNT(*) AS visit_count
FROM er_visits
GROUP BY patient_outcome
ORDER BY avg_wait_time DESC;
"""

# run query and load into pandas
outcome_df = pd.read_sql_query(outcome_query, conn)
print(outcome_df)

           patient_outcome  avg_wait_time  visit_count
0  Left Without Being Seen         179.05          253
1               Discharged          94.96         2879
2                 Admitted          48.65         1868
